In [82]:
import numpy as np

In [83]:
set = '2407_I'
star_id = 233


reference_fit = f'NGC6426_{set[2:]}/ref.fits'
star_lc = f'NGC6426_{set[2:]}/lcs/lc{str(star_id)}.data'
physical_coords_file = f'NGC6426_{set[2:]}/phot_{set}.data'

In [84]:
lista_sets = !ls NGC6426_* -d

lista_sets

['NGC6426_2407_I',
 'NGC6426_2407_V',
 'NGC6426_2505_I',
 'NGC6426_2505_V',
 'NGC6426_2506A_I',
 'NGC6426_2506A_V',
 'NGC6426_2506B_I',
 'NGC6426_2506B_V',
 'NGC6426_2508_I',
 'NGC6426_2508_V']

In [85]:
for set in lista_sets: 
    print(set)

NGC6426_2407_I
NGC6426_2407_V
NGC6426_2505_I
NGC6426_2505_V
NGC6426_2506A_I
NGC6426_2506A_V
NGC6426_2506B_I
NGC6426_2506B_V
NGC6426_2508_I
NGC6426_2508_V


In [86]:
!ls '{lista_sets[1]}/lcs/'

lc1000.data  lc1216.data  lc166.data  lc357.data  lc584.data  lc772.data
lc1001.data  lc1217.data  lc167.data  lc358.data  lc585.data  lc773.data
lc1002.data  lc1218.data  lc168.data  lc359.data  lc586.data  lc774.data
lc1003.data  lc1219.data  lc169.data  lc35.data   lc588.data  lc775.data
lc1004.data  lc121.data   lc16.data   lc361.data  lc589.data  lc776.data
lc1005.data  lc1221.data  lc170.data  lc362.data  lc58.data   lc777.data
lc1006.data  lc1222.data  lc171.data  lc363.data  lc590.data  lc778.data
lc1007.data  lc1223.data  lc172.data  lc365.data  lc591.data  lc779.data
lc1008.data  lc1224.data  lc173.data  lc366.data  lc592.data  lc77.data
lc1009.data  lc1225.data  lc174.data  lc367.data  lc593.data  lc780.data
lc100.data   lc1226.data  lc175.data  lc368.data  lc594.data  lc781.data
lc1010.data  lc1227.data  lc176.data  lc369.data  lc595.data  lc782.data
lc1011.data  lc1228.data  lc177.data  lc36.data   lc596.data  lc783.data
lc1013.data  lc1229.data  lc178.data  lc370.data  lc

In [87]:
from to_magnitudes import get_physical_by_id
from to_magnitudes import get_m_ref
from to_magnitudes import isis_flux_to_mag
import pandas as pd 
import matplotlib.pyplot as plt

def main(set, star_id, reference_fit, star_lc, physical_coords_file): 
    delta_flux = np.loadtxt(star_lc, dtype=str)[:,1]
    delta_flux = delta_flux.astype(np.float64)
    
    x_physical, y_physical = get_physical_by_id(physical_coords_file, star_id)
    
    m_ref, flux_ref = get_m_ref(reference_fit, x_physical, y_physical)
    print(f"m_ref = {m_ref:.3f} (flux = {flux_ref:.1f})")
    
    mag_instr = isis_flux_to_mag(delta_flux, m_ref, C0=25)
    
    # NEW
    new_lc = pd.read_csv(star_lc, delim_whitespace=True, header=None)
    new_lc[6] = mag_instr
    
    #plt.scatter(new_lc[0], new_lc[6], s=4)
    #plt.show()  # ¡Añade esto para ver el plot!
    
    # CORRECCIÓN: Guardar con extensión .data y en el path correcto
    output_path = f'{set}/lcs/lc{star_id}.data'  # ¡Mantén la extensión!
    new_lc.to_csv(output_path, sep="\t", index=False, header=False)
    print(f"✅ Guardado: {output_path}")
    

In [88]:
variable_ids = np.loadtxt('NGC6426/Variables.dat', dtype=str)[:,0]

variable_ids.astype(np.str)

/home/astro/miniconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


array(['118', '121', '167', '201', '217', '233', '265', '432', '556',
       '572', '604', '688', '842', '940', '950', '961', '1059', '1061',
       '1233', '1278'], dtype='<U26')

In [89]:
import os
import glob

# Primero obtenemos la lista de sets (directorios NGC6426_*)
lista_sets = glob.glob('NGC6426_*/')
lista_sets = [s.rstrip('/') for s in lista_sets]  # Remover la barra final

print(f"Sets encontrados: {lista_sets}")

for set_dir in lista_sets:
    # Construir ruta completa a la carpeta lcs
    lcs_path = os.path.join(set_dir, 'lcs')
    
    # Verificar que la carpeta existe
    if not os.path.exists(lcs_path):
        print(f"⚠️  No existe {lcs_path}, saltando...")
        continue
        
    # Listar archivos lc*.data en la carpeta lcs
    lc_files = [f'lc{star_id}.data' for star_id in variable_ids]
    
    print(f"📁 Procesando {set_dir} ({len(lc_files)} archivos lc)")
    
    for lc_file in lc_files:
        # Extraer star_id del nombre del archivo (ej: "lc123.data" → "123")
        base_name = os.path.basename(lc_file)
        star_id = base_name.replace('lc', '').replace('.data', '')
        
        # Construir paths completos (AJUSTA según tu estructura)
        reference_fit = f'{set_dir}/ref.fits'
        physical_coords_file = f'{set_dir}/phot_{set_dir.replace("NGC6426_", "")}.data'
        
        # Verificar que los archivos necesarios existen
        if not all(os.path.exists(f) for f in [reference_fit, physical_coords_file]):
            print(f"   ⚠️  Archivos faltantes para star {star_id}, saltando...")
            continue
        
        print(f"   ⭐ Procesando star_id: {star_id}")
        
        try:
            # Llamar a tu función main con los parámetros CORRECTOS
            main(set=set_dir, 
                 star_id=star_id, 
                 reference_fit=reference_fit, 
                 star_lc=lc_file, 
                 physical_coords_file=physical_coords_file)
            
        except Exception as e:
            print(f"   ❌ Error procesando star {star_id}: {str(e)}")
            continue

print("✅ Procesamiento completado")

Sets encontrados: ['NGC6426_2506A_V', 'NGC6426_2508_V', 'NGC6426_2505_V', 'NGC6426_2407_V', 'NGC6426_2505_I', 'NGC6426_2508_I', 'NGC6426_2506B_I', 'NGC6426_2506B_V', 'NGC6426_2506A_I', 'NGC6426_2407_I']
📁 Procesando NGC6426_2506A_V (20 archivos lc)
   ⭐ Procesando star_id: 118
   ❌ Error procesando star 118: lc118.data not found.
   ⭐ Procesando star_id: 121
   ❌ Error procesando star 121: lc121.data not found.
   ⭐ Procesando star_id: 167
   ❌ Error procesando star 167: lc167.data not found.
   ⭐ Procesando star_id: 201
   ❌ Error procesando star 201: lc201.data not found.
   ⭐ Procesando star_id: 217
   ❌ Error procesando star 217: lc217.data not found.
   ⭐ Procesando star_id: 233
   ❌ Error procesando star 233: lc233.data not found.
   ⭐ Procesando star_id: 265
   ❌ Error procesando star 265: lc265.data not found.
   ⭐ Procesando star_id: 432
   ❌ Error procesando star 432: lc432.data not found.
   ⭐ Procesando star_id: 556
   ❌ Error procesando star 556: lc556.data not found.
   ⭐

In [ ]:
from to_magnitudes import get_physical_by_id, get_m_ref, isis_flux_to_mag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def flux_to_magnitudes(set_name, star_id, reference_fit, star_lc, physical_coords_file): 
    # leer archivo con pandas (más robusto que np.loadtxt)
    df = pd.read_csv(star_lc, delim_whitespace=True, header=None)

    # delta_flux está en la columna 1 (segunda columna)
    delta_flux = df[1].to_numpy(dtype=float)
    
    # coordenadas físicas
    x_physical, y_physical = get_physical_by_id(physical_coords_file, star_id)
    
    # magnitud de referencia
    m_ref, flux_ref = get_m_ref(reference_fit, x_physical, y_physical)
    print(f"m_ref = {m_ref:.3f} (flux = {flux_ref:.1f})")
    
    # conversión a magnitudes instrumentales
    mag_instr = isis_flux_to_mag(delta_flux, m_ref, C0=25)
    
    # agregar magnitud como nueva columna
    df[6] = mag_instr
    
    # opcional: visualizar
    plt.scatter(df[0], df[6], s=4)
    plt.gca().invert_yaxis()
    plt.show()
    
    # guardar en mismo set
    output_path = f'{set_name}/lcs/lc{star_id}_mag.data'
    df.to_csv(output_path, sep="\t", index=False, header=False)
    print(f"✅ Guardado: {output_path}")
    return df
